In [5]:
import os
import pandas as pd 
import numpy as np
import operator
from ast import literal_eval
import gzip 

### run by python3

os.chdir('~/Downloads/')


def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')



### Create dataframe for both meta-data and reviews  
df1 = getDF('amazonproject/meta_health_strict.json.gz') 

df2 = getDF('amazonproject/reviews_Health_and_Personal_Care.json.gz')  

print("This is the first line of df1:")
print(df1.head(1))
print("=========================================")
print("This is the first line of df2:")
print(df2.head(1))
print('finish')


This is the first line of df1:
         asin                              description  \
0  0077614992  This is an example product description.   

                                     title  \
0  Principles of Mgmt + Oper-CSUF Custom C   

                                               imUrl  \
0  http://ecx.images-amazon.com/images/I/51G%2BRq...   

                                             related  \
0  {'also_bought': ['0471730726', '0132834871', '...   

                            salesRank                  categories  price brand  
0  {'Health & Personal Care': 168429}  [[Health & Personal Care]]    NaN   NaN  
This is the first line of df2:
      reviewerID        asin reviewerName helpful  \
0  ARMDSTEI0Z7YW  0077614992         dodo  [0, 0]   

                                          reviewText  overall summary  \
0  This book was a requirement for a college clas...      5.0   great   

   unixReviewTime   reviewTime  
0      1360886400  02 15, 2013  
finish


In [6]:
### Obtain ReviewText, ReviewSummary from df2.
def getfromdf2(item):
   list1 = df2[(df2.ix[:, 1] == idx)]['reviewText'].fillna('').values.tolist()
   str1 = ' '.join(str(u) for u in list1)
   return str1;


f = open('amazon-easy-health.csv','w')


for i in range(0,len(df1['asin'])):
    
    idx = str(df1['asin'][i])
    des = df1['description'][i]
    category = df1['categories'][i]
    title = df1['title'][i]
    price = df1['price'][i]
    
    
   
    
    textstring = getfromdf2('reviewText') 
    sumstring =  getfromdf2('summary')
    votestring = getfromdf2('helpful')
    
 

    ### Review score calculation
    scorelist = df2[(df2.ix[:, 1] == idx)]['overall'].fillna('').values.tolist()
    if(len(scorelist)==0):
           ave = 'no' 
    else:
           ave = sum(scorelist)/float(len(scorelist))
        
    


    ####Sales Rank
    if(pd.isnull(df1['salesRank'][i])):
             salekey = 'no'
             salevalue = 'no'
    else:     
             salekey = list(df1['salesRank'][i])[0]
             salevalue = list(df1['salesRank'][i].values())[0]
    
  
    
    
    
    s = ( str(title) + ' ****** ' + str(des) + ' ****** ' + str(textstring) + ' ****** ' 
          + str(sumstring) + ' ****** ' + str(price) + ' ****** ' + str(ave) + ' ****** '
          + str(category) + ' ****** ' + str(salekey) + ' ****** ' + str(salevalue) )
    
    
    s = s.replace('\n', ' ')
   
   
    
    f.write(id + "\t" + s + "\n")
    

f.close()  